# TRAIN notebook
cometition : RSNA Screening Mammography Breast Cancer Detection  
url : https://www.kaggle.com/competitions/rsna-breast-cancer-detection  

In [3]:
def transform_image(paths, side='left', size=512, threshold=0.05):
    dicom_data = pydicom.dcmread(paths)
    data = np.array(dicom_data.pixel_array)
    data = data - np.min(data)
    data = data / np.max(data)
    if dicom_data.PhotometricInterpretation == "MONOCHROME1":
        data = 1.0 - data
    image = data[5:-5, 5:-5]

    ret, thresh = cv2.threshold(image, threshold, 1, 0)

    width = image.shape[1]
    # take all columns up to half image (in width), sumarize them and compare with other half
    if sum(sum(thresh[:, :width // 2])) > sum(sum(thresh[:, width // 2:])): 
        image_side = 'left'
    else:
        image_side = 'right'

    if image_side != side: 
        image = cv2.flip(image, 1)
    output= cv2.connectedComponentsWithStats((image > 0.05).astype(np.uint8)[:, :], 8, cv2.CV_32S)
    stats = output[2] # left, top, width, height, area_size

    idx = stats[1:, 4].argmax() + 1
    x1, y1, w, h = stats[idx][:4]
    x2 = x1 + w
    y2 = y1 + h

    image = image[y1: y2, x1: x2]
    image = cv2.resize(image, (size, size))
    return image

class EarlyStopping:
    def __init__(self, patience=3, verbose=False, path='checkpoint_model.pth'):
        self.patience = patience #設定ストップカウンタ
        self.verbose = verbose #表示の有無
        self.counter = 0 #現在のカウンタ値
        self.best_score = None #ベストスコア
        self.early_stop = False #ストップフラグ
        self.val_loss_min = np.Inf # 前回のベストスコア記憶用
        self.path = path #ベストモデルの格納パス
    
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.checkpoint(val_loss, model)
            self.counter = 0
    
    def checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'validation loss decreased({self.val_loss_min:.6f} ---> {val_loss:.6f}). saving model....')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


In [4]:
class TrainDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        f = self.df.filename.tolist()[index]
        image = transform_image(f)
        target = torch.Tensor(self.df.cancer.tolist())[index]
        image = torch.Tensor(image)
        send = {'image': image, 'target': target}
        return send

In [5]:
class Config:
    epochs = 10
    batch_size = 32
    size = 512
    seed = 0
config = Config()

# 2023/2/13

## import

In [1]:
!pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm

In [18]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import cv2
import PIL
import pydicom
import gdcm
import pylibjpeg
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tqdm
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
train['filename'] = train.apply(lambda x: '/kaggle/input/rsna-breast-cancer-detection/train_images/'+str(x.patient_id)+'/'+str(x.image_id)+'.dcm', axis=1)

In [7]:
class Resnet26d(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('resnest26d', pretrained=True, in_chans=1, num_classes=0)
        self.in_features = self.backbone.num_features
        
        self.head = nn.Sequential(
            nn.Linear(self.in_features, 100),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        h = self.backbone(x)
        y = self.head(h)
        return y
        

In [8]:
def balance_df(df):
    one_x = df[df['cancer']==1]
    zero_x = df[df['cancer']==0]
    zero_x_2 = zero_x.sample(one_x.shape[0], random_state=0)
    new_df = pd.concat([one_x, zero_x_2])
    new_df = new_df.reset_index()
    return new_df

In [9]:
new_df = balance_df(train.copy())

In [10]:
train, test = train_test_split(new_df, stratify=new_df.cancer, random_state=config.seed, test_size=0.2)
dataset = TrainDataset(train)
dataloader = DataLoader(dataset, batch_size=config.batch_size, drop_last=True, shuffle=True)

In [24]:
def train_net(n_epochs, train_loader, net, optimizer, loss_fn, device='cpu'):
    earlystopping = EarlyStopping(verbose=True)
    print(f'device=={device}')
    losses = []
    net.to(device)
#     torch.manual_seed(config.seed)
    torch.cuda.manual_seed(config.seed)
    
    for epoch in range(n_epochs):
        running_loss = 0
        net.train()
        print(f'epoch {epoch} start')
        for index, data in enumerate(tqdm.tqdm(train_loader)):
            inputs, labels = data['image'], data['target']
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.unsqueeze(dim=1)
            labels = labels.unsqueeze(dim=1)
            
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = loss_fn(outputs, labels)
            print(f'index: {index}, loss: {loss}')
            print(f'outputs: {outputs.tolist()[:3]}, labels: {labels.tolist()[:3]}')
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        losses.append( running_loss / index)
        print(f'epoch, {epoch} : {running_loss/ index}')
        
        earlystopping((running_loss/index), net)
        if earlystopping.early_stop:
            print('Early Stop!!!!!')
            break
    return losses

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
net = Resnet26d()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.005)

In [ ]:
losses = train_net(n_epochs=config.epochs, train_loader=dataloader,  net=net, optimizer=optimizer, loss_fn=criterion, device=device)

device==cuda
epoch 0 start


  0%|          | 0/57 [00:00<?, ?it/s]

index: 0, loss: 0.6960254907608032
outputs: [[0.4700239598751068], [0.49325302243232727], [0.4398552477359772]], labels: [[0.0], [1.0], [1.0]]


  2%|▏         | 1/57 [00:30<28:38, 30.69s/it]

index: 1, loss: 0.6746928691864014
outputs: [[0.4767273962497711], [0.4542611837387085], [0.48463010787963867]], labels: [[1.0], [0.0], [0.0]]


...

 61%|██████▏   | 35/57 [19:00<11:53, 32.45s/it]

index: 35, loss: 0.676628589630127
outputs: [[0.5359988212585449], [0.501485288143158], [0.45624756813049316]], labels: [[0.0], [1.0], [0.0]]


 63%|██████▎   | 36/57 [19:26<10:42, 30.57s/it]

index: 36, loss: 0.7174524068832397
outputs: [[0.4717674255371094], [0.4915623068809509], [0.5333186984062195]], labels: [[0.0], [0.0], [0.0]]


 65%|██████▍   | 37/57 [20:00<10:32, 31.62s/it]

In [ ]:
test_dataset = TrainDataset(test)
test_dataloader = DataLoader(test_dataset, batch_size=32)
model = Resnet26d()
model.load_state_dict(torch.load('/kaggle/working/checkpoint_model.pth'))

def test_net(dataloader, model, device):
    model = model.eval()
    preds_lis = []
    labels_lis = []
    model.to(device)
    for i, data in enumerate(dataloader):
        with torch.no_grad():
            inputs, labels = data['image'], data['target']
            inputs = inputs.to(device)
            labels = labels.to(device)
            inputs = inputs.unsqueeze(dim=1)
            labels = labels.unsqueeze(dim=1)
            inputs.to(device)
            
            output = model(inputs)
            preds_lis.append(output)
            labels_lis.append(labels)
    return preds_lis, labels_lis

preds, labels = test_net(test_dataloader, model, device)

def convert(lists):
    tmp = []
    for i in range(len(lists)):
        x = lists[i].tolist()
        for j in range(len(x)):
            tmp.append(x[j][0])
    return tmp

preds = convert(preds)
labels = convert(labels)

def pfbeta(labels, predictions, beta = 1):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    
pfbeta(labels, preds)